[![Labellerr](https://storage.googleapis.com/labellerr-cdn/%200%20Labellerr%20template/notebook.webp)](https://www.labellerr.com)

# **SAM3 Batch Inference**

---

[![labellerr](https://img.shields.io/badge/Labellerr-BLOG-black.svg)](https://www.labellerr.com/blog)
[![Youtube](https://img.shields.io/badge/Labellerr-YouTube-b31b1b.svg)](https://www.youtube.com/@Labellerr)
[![Github](https://img.shields.io/badge/Labellerr-GitHub-green.svg)](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)

Perform batch inference on images using SAM3 (Segment Anything Model v3) with text prompts and export annotations in COCO-JSON format. Seamlessly upload results to Labellerr for review and refinement.

## **Quickstart**

1. **Install dependencies** — Follow the [README.md](README.md) to set up SAM3 and Labellerr SDK

2. **Download SAM3 model** — Get `sam3.pt` from [Hugging Face](https://huggingface.co/facebook/sam3) and place it in the `model/` folder

3. **Configure inference** — Set `INPUT_FOLDER` (your images), `MODEL_CHECKPOINT`, and `TEXT_PROMPTS` (objects to detect)

4. **Run batch inference** — Execute `run_batch_inference()` to generate COCO-JSON annotations in `SAM3_Results/`

5. **Upload to Labellerr** — Set your `PROJECT_ID` and run `upload_preannotations()` to review annotations on [Labellerr](https://www.labellerr.com)

## **INSTALLATION**

Run the below cell and follow the instructions in the [README.md](README.md)

In [ ]:
# uncomment the following lines if you have not cloned the repository and installed the SDK
# !git clone https://github.com/facebookresearch/sam3.git
# %pip install git+https://github.com/Labellerr/SDKPython.git

## **1. Download SAM3 Model from Huggingface**


### SAM3 Batch Inference: Model Download Instructions

To perform SAM3 batch inference, you need to download the SAM3 model from the Hugging Face model hub.

**SAM 3 (Segment Anything Model v3)** is developed by Meta and currently requires access permission.

### Steps to get the SAM3 Model:

1.  **Step 1:** Visit the official [SAM 3 repository on Hugging Face](https://huggingface.co/facebook/sam3).
2.  **Step 2:** Request access (log in to Hugging Face and fill out the access form).
3.  **Step 3:** Once you’re approved, download the model checkpoint from files and versions (e.g., `sam3.pt`) in the designated model folder.

### **Import Required functions**

In [1]:
# This function is responsible for executing the batch inference process using the SAM3 model,
from sam3_batch_inference import run_batch_inference

# This function is designed to handle the batch uploading of pre-generated annotations to a Labellerr project.
from batch_upload_preannot import upload_preannotations

## **2. Fill the parameters for SAM3 batch inference**

In [2]:
# ============================================================
# CONFIGURATION - Fill this values
# ============================================================

# Path to the folder containing input images.
# Example: "flower_sample_img"
INPUT_FOLDER = "sample_img"

# Path to the pre-trained model checkpoint file (.pt).
# This file contains the weights for the SAM3 model.
# Example: r"model/sam3.pt"
MODEL_CHECKPOINT = "model/sam3.pt"

# A list of text prompts to be used for segmenting objects within the images.
# Each string in the list represents a different object or class to detect.
# Example: ["Red Flower", "Yellow Flower", "White Flower", "Violet Flower"]
TEXT_PROMPTS = ["Car", 
                "Bus", 
                "Person",
                ]

In [ ]:
# Call the batch inference function with specified parameters.
run_batch_inference(
    INPUT_FOLDER,       # Path to the input folder containing images for inference.
    TEXT_PROMPTS,       # List of text prompts to guide the segmentation model.
    MODEL_CHECKPOINT,   # Path to the pre-trained model checkpoint file (.pt).
    
)

## **4. Labellerr Annotation Review Workflow**

In [3]:
from labellerr.client import LabellerrClient
from dotenv import load_dotenv
import os

# For Labellerr Dataset 
from labellerr.core.datasets import create_dataset_from_local, LabellerrDataset
from labellerr.core.schemas import DatasetConfig

# For Annotation Template
from labellerr.core.annotation_templates import create_template
from labellerr.core.schemas.annotation_templates import AnnotationQuestion, QuestionType, CreateTemplateParams, DatasetDataType
from template_helper import create_questions_from_prompts
import uuid

# For labellerr Project
from labellerr.core.projects import create_project, LabellerrProject
from labellerr.core.schemas.projects import CreateProjectParams, RotationConfig

## **Configuration Setup steps**

1. [Go to Labellerr](https://www.labellerr.com/?utm_source=githubY&utm_medium=social&utm_campaign=github_clicks) and Login.

2. From the Labellerr API tab [check the docs](https://docs.labellerr.com/sdk/getting-started#getting-started), copy the `API Key`, `API SECRET`, and `CLIENT ID`. Then, create a `.env` file and paste the values into it as follows: 

    ```
    API_KEY = ""
    API_SECRET = ""
    CLIENT_ID = ""
    ```

    **NOTE:** Ensure the `.env` file is in the same directory as this notebook, and the variable names match the strings shown above exactly.

3. Fill the values in the cells below.

### ***FILL THESE VALUES***

In [4]:
# Copy the path of the result annotations folder (where annotations are stored) and paste it into the `ANNOTATION_DIR` variable.
ANNOTATION_DIR = "SAM3_Results/sample_img"


DATASET_NAME = "My_Labellerr_Dataset"
TEMPLATE_NAME = "SDK SAM3 Batch Inference Template"
PROJECT_NAME = "SDK SAM3 Review Project"

In [5]:
load_dotenv(".env")

client_id = os.getenv("CLIENT_ID")
api_key = os.getenv("API_KEY")
api_secret = os.getenv("API_SECRET")

if not client_id or not api_key or not api_secret:
    raise ValueError("CLIENT_ID, API_KEY, and API_SECRET must be set in the .env file")

CLIENT = LabellerrClient(client_id=client_id, 
                        api_key=api_key, 
                        api_secret=api_secret)

### **4.1. Creation Annotation Guideline Based on your Text Prompts**

In [8]:
QUESTIONS = create_questions_from_prompts(TEXT_PROMPTS)
template = create_template(
    client=CLIENT,
    params=CreateTemplateParams(
        template_name=TEMPLATE_NAME,
        data_type=DatasetDataType.image,
        questions=QUESTIONS
    )
)

if template.annotation_template_id is not None:
    print(f"Template created successfully with ID: {template.annotation_template_id}")
else:
    print("Failed to create template")

Template created successfully with ID: 63c14a2c-4594-4df7-b356-1fc9960c255a


### **4.1. Create a Dataset on labellerr using your local images folder**

In [ ]:
dataset = create_dataset_from_local(
        client=CLIENT,
        dataset_config=DatasetConfig(dataset_name=DATASET_NAME, 
                                     data_type="image"),
        folder_to_upload=INPUT_FOLDER,
    )

print("Waiting for dataset to be processed")
dataset.status()
print("Dataset processed")

print("Dataset ID: ", dataset.dataset_id)

### **4.2. Create a Labellerr Annotation Project using that Dataset**

In [ ]:
project = create_project(
    client=CLIENT,
    params=CreateProjectParams(
        project_name=PROJECT_NAME,
        data_type=DatasetDataType.image,
        rotations=RotationConfig(
            annotation_rotation_count=1,
            review_rotation_count=1,
            client_review_rotation_count=1
        )
    ),
    datasets=[dataset],
    annotation_template=template
)

print("Created project with ID: " + project.project_id)

### **4.3. Upload SAM3 Pre-Annotations on labellerr**

In [14]:
result = upload_preannotations(
    project_id=project.project_id,
    annotation_format='coco_json',
    batch_annotation_dir=ANNOTATION_DIR
)

Batch Upload Started
Project ID: eryn_defeated_pony_99911
Annotation Format: coco_json
Total batch files: 1
Delay between uploads: 2s

[1/1] Uploading: annotations.json
  ✓ Success! Metadata: {'questions_ignored': [], 'activity_id': 'c5c47abf-b37b-4ec9-83ea-05a3df3ae4a6', 'files_not_updated': ['traffic_3_frame_000965_t16.08s_000149.jpg', 'traffic_3_frame_000738_t12.30s_000139.jpg']}

Batch Upload Complete!
Total batches: 1
Successful uploads: 1
Failed uploads: 0
Total time: 113.77s


## **5. Review SAM3 annotations on Labellerr**

### When Pre-Annotations are loaded, you can now go to [Labellerr UI](https://www.labellerr.com/?utm_source=githubY&utm_medium=social&utm_campaign=github_clicks) to review and fix the annotations

## **6. Export your annotations from Labellerr UI**

![Labellerr](https://storage.googleapis.com/labellerr-cdn/%200%20Documentation-template/export_guide.webp)

## **7. Train Models with Exported COCO-JSON on Our CV Notebooks**

Check out these hands-on notebooks to train YOLO models for your specific use case:

- **[Fine-Tune your custom data on YOLOv12](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision/blob/main/fine-tune%20YOLO%20for%20various%20use%20cases/Pill_Counting_Using_YOLOv12.ipynb)**  
  Build an automated pill counting system for pharmaceutical and healthcare applications.

- **[Intrusion Detection Using YOLOv8](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision/blob/main/fine-tune%20YOLO%20for%20various%20use%20cases/Intrusion_detection_using_YOLO.ipynb)**  
  Create a security system that detects unauthorized access in real-time.

- **[Fine-Tune RT-DETR for Object Detection](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision/blob/main/fine-tune%20YOLO%20for%20various%20use%20cases/Oil_Field_Monitoring_using_CV.ipynb)**  
  Monitor and analyze vehicle traffic patterns for smart city applications.

- **[Cell Counting with YOLO](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision/blob/main/fine-tune%20YOLO%20for%20various%20use%20cases/Fine-Tune-YOLO-for-Cell-Counting.ipynb)**  
  Automate biological cell counting for medical research and diagnostics.

---

## 🧑‍🔬 Check Our Videos tutorials for Training

Whether you're a beginner or a practitioner, our hands-on training videos are perfect for learning custom model building, computer vision techniques, and applied AI:

- [How to Fine-Tune YOLO on Custom Dataset](https://www.youtube.com/watch?v=pBLWOe01QXU)  
  Step-by-step guide to fine-tuning YOLO for real-world use—environment setup, annotation, training, validation, and inference.
- [Build a Real-Time Intrusion Detection System with YOLO](https://www.youtube.com/watch?v=kwQeokYDVcE)  
  Create an AI-powered system to detect intruders in real time using YOLO and computer vision.
- [Finding Athlete Speed Using YOLO](https://www.youtube.com/watch?v=txW0CQe_pw0)  
  Estimate real-time speed of athletes for sports analytics.
- [Object Counting Using AI](https://www.youtube.com/watch?v=smsjBBQcIUQ)  
  Learn dataset curation, annotation, and training for robust object counting AI applications.

---

## 🤝 Stay Connected

- **Website:** [https://www.labellerr.com/](https://www.labellerr.com/)
- **Blog:** [https://www.labellerr.com/blog/](https://www.labellerr.com/blog/)
- **GitHub:** [Labellerr/Hands-On-Learning-in-Computer-Vision](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)
- **LinkedIn:** [Labellerr](https://in.linkedin.com/company/labellerr)
- **Twitter/X:** [@Labellerr1](https://x.com/Labellerr1)

*Happy learning and building with Labellerr!*